In [1]:
!kaggle competitions download -c playground-series-s3e14


  0%|          | 0.00/634k [00:00<?, ?B/s]
100%|##########| 634k/634k [00:01<00:00, 477kB/s]
100%|##########| 634k/634k [00:01<00:00, 476kB/s]


In [11]:
import os
os.chdir(r"C:\Users\Sanjay Sane\DAI_Oct24\playground-series-s3e14")

In [19]:
from sklearn.ensemble import BaggingRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
import warnings

In [21]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv',index_col=0)

In [23]:
train.columns

Index(['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia',
       'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange',
       'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange',
       'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds',
       'yield'],
      dtype='object')

In [25]:
X = train.drop('yield', axis=1)
y = train['yield']

In [33]:
dtr = DecisionTreeRegressor(random_state=24)
bagg = BaggingRegressor(estimator=dtr, random_state=24)
kfold = KFold(n_splits=5, shuffle=True, random_state=24)
params = {'n_estimators':[10,20,50]}
gcv = GridSearchCV(bagg,param_grid=params,cv=kfold,scoring='r2')
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor(random_state=24),
                                        random_state=24),
             param_grid={'n_estimators': [10, 20, 50]}, scoring='r2')

In [34]:
gcv.best_params_,gcv.best_score_

({'n_estimators': 50}, 0.8080941691743722)

In [35]:
best_model = gcv.best_estimator_
y_pred = best_model.predict(test)
y_pred[y_pred<0]

array([], dtype=float64)

In [39]:
best_model

BaggingRegressor(estimator=DecisionTreeRegressor(random_state=24),
                 n_estimators=50, random_state=24)

In [41]:
submit = pd.read_csv('sample_submission.csv')
submit['yield'] = y_pred
submit.to_csv('sbt_bagg_dtr.csv',index=False)

In [53]:
!kaggle competitions submit -c playground-series-s3e14 -f "sbt_bagg_dtr.csv" -m "{'n_estimators': 50}"

Successfully submitted to Regression with a Wild Blueberry Yield Dataset



  0%|          | 0.00/238k [00:00<?, ?B/s]
  7%|6         | 16.0k/238k [00:00<00:02, 94.3kB/s]
100%|##########| 238k/238k [00:01<00:00, 131kB/s]  


In [57]:
!kaggle competitions submissions playground-series-s3e14

fileName             date                 description                                          status    publicScore  privateScore  
-------------------  -------------------  ---------------------------------------------------  --------  -----------  ------------  
sbt_bagg_dtr.csv     2024-11-09 10:18:27  {'n_estimators': 50}                                 complete  373.17072    361.56485     
sbt_bagg_scl_lr.csv  2024-11-09 09:29:26  {'BAGG__n_estimators': 20, 'SCL': StandardScaler()}  complete  368.68694    363.65289     
